In [40]:
#Universidade Federal de Pernambuco
#Disciplina: Introdução a Microssimulação
#Professor: Jose Ricardo Nogueira
#Aluno: Paulo Francisco da Silva Junior

In [41]:
#importações
import pandas as pd
from numpy import nan

Calculdo da contribuição à previdência com base nas regras legais de 2019

In [42]:
#Importando banco
pnadc = pd.read_csv("/content/drive/MyDrive/Microsimulação/PNADC_2019_mine.csv",sep=",",decimal=".",dtype=float,na_values=" ")

In [43]:
#Corrigindo missings pra 0
pnadc.fillna(0,inplace=True)

Considerando que os valores dos salários primários e secundários presentes na PNADC estão mensalizados, o 13 salário e o adicional de férias serão divididos por caso a análise seja anualizada (o correto a se fazer).

In [44]:
# ABONO SALARIAL
# OBS: A simulacao e realizada sem utilizacao da restricao de tempo de trabalho.
# OBS: Em substituicao  a restricao de tempo, usa-se uma restricao de idade (>=23 anos), 
# de forma a excluir os possiveis "aprendizes" (idade entre 16-18 anos), os quais nao tem direito ao beneficio.
# OBS: A diferenca entre 23 anos e 18 anos seria equivalente ao requisito de 5 anos de inscricao no PIS/PASEP.

In [45]:
# ABONO SALARIAL (TRABALHO PRINCIPAL)
pnadc.loc[((pnadc["V2009"]>=23) & (pnadc["VD4009"] == 1) | (pnadc["VD4009"] == 5) | (pnadc["VD4009"] == 7)) &  \
          ((pnadc["V403312"]>=0) & (pnadc["V403312"])<=2*998), 
          "ABOSAL1"] = 998.00 / 12
pnadc.fillna(value={"ABOSAL1":0},inplace=True)

In [46]:
# ABONO SALARIAL (TRABALHO SECUNDARIO)
pnadc.loc[(pnadc["V2009"]>=23) & \
          (((pnadc["V4043"]==4) & (pnadc["V4047"]==1)) | \
           ((pnadc["V4043"]==4) & (pnadc["V4048"]==1)) | \
           (pnadc["V4043"]==2) | \
           ((pnadc["V4043"]==3) & (pnadc["V4048"]==1))) & \
           ((pnadc["V405012"]>=0) & (pnadc["V405012"]<=2*998.00)),
          "ABOSAL2"] = 998.00 / 12
pnadc.fillna(value={"ABOSAL2":0},inplace=True)

In [47]:
# ABONO SALARIAL (TRABALHO PRINCIPAL + SECUNDARIO)
pnadc["ABOSAL"] = pnadc["ABOSAL1"]+pnadc["ABOSAL2"]

In [48]:
# ABONO ANUAL (DECIMO TERCEIRO SALARIO DO APOSENTADO E DO PENSIONISTA)

In [49]:
# APOSENTADORIA E PENSAO
pnadc.loc[pnadc["V5004A2"]>0, "ABONOA"] = pnadc["V5004A2"] / 12
pnadc.fillna(value={"ABONOA":0},inplace=True)

In [50]:
# DECIMO TERCEIRO SALARIO

In [51]:
# DECIMO TERCEIRO SALARIO (TRABALHO PRINCIPAL)
pnadc.loc[(pnadc["VD4009"]==1) | (pnadc["VD4009"]==3) | (pnadc["VD4009"]==5) | (pnadc["VD4009"]==7), 
          "SAL13_1"] = (pnadc["V403312"]-pnadc["SALFAM1"]) / 12
pnadc.loc[pnadc["SAL13_1"]<0,"SAL13_1"]=0
pnadc.fillna(value={"SAL13_1":0},inplace=True)

In [52]:
# DECIMO TERCEIRO SALARIO (TRABALHO SECUNDARIO)
pnadc.loc[((pnadc["V4043"]==4) & (pnadc["V4047"]==1)) | \
          ((pnadc["V4043"]==4) & (pnadc["V4048"]==1)) | \
          (pnadc["V4043"]==2) | \
          ((pnadc["V4043"]==3) & (pnadc["V4048"]==1)) | \
          ((pnadc["V4043"]==1) & (pnadc["V4048"]==1)),
          "SAL13_2"] = (pnadc["V405012"]-pnadc["SALFAM2"]) / 12

pnadc.loc[pnadc["SAL13_2"]<0,"SAL13_2"]=0
pnadc.fillna(value={"SAL13_2":0},inplace=True)

In [53]:
# DECIMO TERCEIRO SALARIO (TRABALHO PRINCIPAL + SECUNDARIO)
pnadc["SAL13"]=pnadc["SAL13_1"]+pnadc["SAL13_2"]

In [54]:
# ADICIONAL DE FERIAS

In [55]:
# ADICIONAL DE FERIAS (TRABALHO PRINCIPAL)
pnadc.loc[(pnadc["VD4009"]==1) | (pnadc["VD4009"]==3) | (pnadc["VD4009"]==5) | (pnadc["VD4009"]==7),
          "AFERIAS1"] = (1/3) * (pnadc["V403312"] - pnadc["SALFAM1"]) / 12
pnadc.loc[pnadc["AFERIAS1"]<0,"AFERIAS1"]=0
pnadc.fillna(value={"AFERIAS1":0},inplace=True)

In [56]:
# ADICIONAL DE FERIAS (TRABALHO SECUNDARIO)
pnadc.loc[((pnadc["V4043"]==4) & (pnadc["V4047"]==1)) | \
          ((pnadc["V4043"]==4) & (pnadc["V4048"]==1)) | \
          (pnadc["V4043"]==2) | \
          ((pnadc["V4043"]==3) & (pnadc["V4048"]==1)) | \
          ((pnadc["V4043"]==1) & (pnadc["V4048"]==1)),
          "AFERIAS2"] = (1/3) * (pnadc["V405012"] - pnadc["SALFAM2"]) / 12
pnadc.loc[pnadc["AFERIAS2"]<0,"AFERIAS2"]=0
pnadc.fillna(value={"AFERIAS2":0},inplace=True)

In [57]:
# ADICIONAL DE FERIAS (TRABALHO PRINCIPAL E SECUNDARIO)
pnadc["AFERIAS"]=pnadc["AFERIAS1"]+pnadc["AFERIAS2"]

In [58]:
#SIMULACAO DAS CONTRIBUICOES SOCIAIS (PNADC 2019)

In [59]:
#1. EMPREGADO E TRABALHADOR AVULSO 

In [60]:
#1.1 EMPREGADO E TRABALHADOR AVULSO (TRABALHO PRINCIPAL)
pnadc.loc[(pnadc["VD4009"]==1) & \
          ((pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"]) <=1751.81),\
          "CONTE1"] = 0.08 * (pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])

pnadc.loc[(pnadc["VD4009"]==1) & \
          ((pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])> 1751.81) & \
          ((pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])<=2919.72), \
          "CONTE1"] = 0.09 * (pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])

pnadc.loc[(pnadc["VD4009"]==1) & \
          ((pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])> 2919.72) & \
          ((pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])<=5839.45), \
          "CONTE1"] = 0.11 * (pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])

pnadc.loc[(pnadc["VD4009"]==1) & \
          ((pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])> 5839.45), \
          "CONTE1"] = 0.11 * 5839.45


pnadc.fillna(value={"CONTE1":0},inplace=True)

In [61]:
#1.2 EMPREGADO E TRABALHADOR AVULSO (TRABALHO SECUNDARIO)
pnadc.loc[((pnadc["V4043"]==3) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])<=1751.81),
          "CONTE2"] = 0.08 * (pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])

pnadc.loc[((pnadc["V4043"]==3) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])> 1751.81) & \
          ((pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])<=2919.72), \
          "CONTE2"] = 0.09 * (pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])

pnadc.loc[((pnadc["V4043"]==3) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])> 2919.72) & \
          ((pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])<=5839.45), \
          "CONTE2"] = 0.11 * (pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])

pnadc.loc[((pnadc["V4043"]==3) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])> 5839.45), \
          "CONTE2"] = 0.11 * 5839.45

pnadc.fillna(value={"CONTE2":0},inplace=True)

In [62]:
#1.3 EMPREGADO E TRABALHADOR AVULSO (TRABALHO PRINCIPAL + SECUNDARIO)
pnadc["CONTE"]= pnadc["CONTE1"]+pnadc["CONTE2"]
pnadc.fillna(value={"CONTE":0},inplace=True)

In [63]:
#2. EMPREGADO DOMESTICO

In [64]:
#2.1 EMPREGADO DOMESTICO (TRABALHO PRINCIPAL)
pnadc.loc[(pnadc["VD4009"]==3) & ((pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])<=1751.81), \
          "CONTED1"] = 0.08 * (pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])

pnadc.loc[(pnadc["VD4009"]==3) & ((pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])>1751.81) & \
          ((pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])<=2919.72), \
          "CONTED1"] = 0.09 * (pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])

pnadc.loc[(pnadc["VD4009"]==3) & ((pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])>2919.72) & \
          ((pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])<=5839.45), \
          "CONTED1"] = 0.11 *  (pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])

pnadc.loc[(pnadc["VD4009"]==3) & ((pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])>5839.45), \
          "CONTED1"] = 0.11 * 5839.45 

pnadc.fillna(value={"CONTED1":0},inplace=True)

In [65]:
#2.2 EMPREGADO DOMESTICO (TRABALHO SECUNDARIO)
pnadc.loc[((pnadc["V4043"]==1) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])<=1751.81), \
          "CONTED2"] = 0.08 * (pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])

pnadc.loc[((pnadc["V4043"]==1) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])>1751.81) & \
          ((pnadc["V405012"] + pnadc["AFERIAS2"])<=2919.72), \
          "CONTED2"] = 0.09 * (pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])

pnadc.loc[((pnadc["V4043"]==1) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])>2919.72) & \
          ((pnadc["V405012"] + pnadc["AFERIAS2"])<=5839.45), \
          "CONTED2"] = 0.11 * (pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])

pnadc.loc[((pnadc["V4043"]==1) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])>5839.45), \
          "CONTED2"] = 0.11 * 5839.45

pnadc.fillna(value={"CONTED2":0},inplace=True)

In [66]:
#2.3 EMPREGADO DOMESTICO (TRABALHO PRINCIPAL + SECUNDARIO)
pnadc["CONTED"]= pnadc["CONTED1"]+pnadc["CONTED2"]
pnadc.fillna(value={"CONTED":0},inplace=True)

In [67]:
#4. SERVIDOR PUBLICO ESTATUTARIO FEDERAL

In [68]:
#4.1 SERVIDOR PUBLICO ESTATUTARIO FEDERAL (TRABALHO PRINCIPAL)
pnadc.loc[(pnadc["V4012"]==4) & (pnadc["V4028"]==1) & (pnadc["V4014"]==1), \
          "CONTFED1"] = 0.11 * (pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])
          
pnadc.fillna(value={"CONTFED1":0},inplace=True)

In [69]:
#4.2 SERVIDOR PUBLICO ESTATUTARIO FEDERAL (TRABALHO SECUNDARIO)
pnadc.loc[(pnadc["V4043"]==4) & (pnadc["V4047"]==1) & (pnadc["V4045"]==1), \
          "CONTFED2"] = 0.11 * (pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])

pnadc.fillna(value={"CONTFED2":0},inplace=True)

In [70]:
#4.3 SERVIDOR PÚBLICO ESTATUTARIO FEDERAL (TRABALHO PRINCIPAL + SECUNDARIO)
pnadc["CONTFED"]= pnadc["CONTFED1"] + pnadc["CONTFED2"]
pnadc.fillna(value={"CONTFED":0},inplace=True)

In [71]:
#Utilizando os pesos, calculamos a contribuição total de cada grupo

In [72]:
#Valor total de contribuição - EMPREGADO E TRABALHADOR AVULSO (TRABALHO PRINCIPAL)
(pnadc["CONTE1"]*pnadc["V1032"]).sum()

6551350224.778093

In [73]:
#Valor total de contribuição - EMPREGADO E TRABALHADOR AVULSO (TRABALHO SECUNDARIO)
(pnadc["CONTE2"]*pnadc["V1032"]).sum()

27107325.367364153

In [74]:
#Valor total de contribuição - EMPREGADO DOMESTICO (TRABALHO PRINCIPAL)
(pnadc["CONTED1"]*pnadc["V1032"]).sum()

209835406.64377117

In [75]:
#Valor total de contribuição - EMPREGADO DOMESTICO (TRABALHO SECUNDARIO)
(pnadc["CONTED2"]*pnadc["V1032"]).sum()

1414086.7316205062

In [76]:
#Valor total de contribuição - SERVIDOR PUBLICO ESTATUTARIO FEDERAL (TRABALHO PRINCIPAL)
(pnadc["CONTFED1"]*pnadc["V1032"]).sum()

887372499.8576343

In [77]:
#Valor total de contribuição - SERVIDOR PUBLICO ESTATUTARIO FEDERAL (TRABALHO SECUNDARIO)
(pnadc["CONTFED2"]*pnadc["V1032"]).sum()

1145959.1184278475

In [78]:
pnadc.to_csv("/content/drive/MyDrive/Microsimulação/PNADC_2019_S_C.csv",sep=";",date_format=float,decimal=".")